In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (10201 Collins Ave Unit 2401, Bal Harbour, FL ...
2     (800 South Pointe Dr APT 1203, Miami Beach, FL...
3     (6800 Fisher Island Dr #6862, Miami Beach, FL ...
4     (18501 Collins Ave #4102, Sunny Isles Beach, F...
5     (1300 Brickell Bay Dr APT 4401, Miami, FL 3313...
6     (15701 Collins Ave #1902, North Miami Beach, F...
7     (251 Crandon Blvd APT 624, Key Biscayne, FL 33...
8     (3165 Via Abitare Way #4, Miami, FL 33133, USA...
9     (17000 NW 67th Ave APT 303, Hialeah, FL 33015,...
10    (300 South Pointe Dr APT 1001, Miami Beach, FL...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Buyer Agent,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Oceana Bal Harbour\r10201 Collins Ave 2401\rBa...,8/3/2021,2/8/2022,162,"$10,465,000.00",3992,"$2,621.49",Ludmila Bogatov,Ryan Mendell,Oceana Bal Harbour 10201 Collins Ave 2401 Bal ...,Oceana Bal Harbour,10201 Collins Ave 2401 Bal Harbour,"(10201 Collins Ave Unit 2401, Bal Harbour, FL ...","(25.8951782, -80.1231506, 0.0)",25.895178,-80.123151,0.0
2,Apogee\r800 S Pointe Dr 1203\rMiami Beach,6/28/2021,2/8/2022,176,"$8,600,000.00",3103,"$2,771.51",Michael Duchon,Laurent Bensoussan,Apogee 800 S Pointe Dr 1203 Miami Beach,Apogee,800 S Pointe Dr 1203 Miami Beach,"(800 South Pointe Dr APT 1203, Miami Beach, FL...","(25.7677245, -80.13634490000001, 0.0)",25.767724,-80.136345,0.0
3,Fisher Island\r6800 Fisher Island Dr 6862\rFis...,7/21/2021,2/9/2022,19,"$8,550,000.00",3620,"$2,361.88",Dora Puig,Ashley Cusack,Fisher Island 6800 Fisher Island Dr 6862 Fishe...,Fisher Island,6800 Fisher Island Dr 6862 Fisher Island,"(6800 Fisher Island Dr #6862, Miami Beach, FL ...","(25.7623668, -80.14735929999999, 0.0)",25.762367,-80.147359,0.0
4,Turnberry Ocean Club\r18501 Collins Ave 4102\r...,7/9/2021,2/8/2022,202,"$6,000,000.00",3550,"$2,068.97",Diane Cohen,Danny Lemeshev,Turnberry Ocean Club 18501 Collins Ave 4102 Su...,Turnberry Ocean Club,18501 Collins Ave 4102 Sunny Isles Beach,"(18501 Collins Ave #4102, Sunny Isles Beach, F...","(25.9477873, -80.12022019999999, 0.0)",25.947787,-80.120220,0.0
5,Brickell House Condo\r1300 Brickell Bay Dr 440...,11/4/2021,2/11/2022,36,"$5,200,000.00",6521,$797.42,Lourdes Alatriste,Lourdes Alatriste,Brickell House Condo 1300 Brickell Bay Dr 4401...,Brickell House Condo,1300 Brickell Bay Dr 4401 Miami,"(1300 Brickell Bay Dr APT 4401, Miami, FL 3313...","(25.7603908, -80.19034649999999, 0.0)",25.760391,-80.190346,0.0
6,15701 Collins Condo\r15701 Collins Ave 1902\rS...,7/27/2021,2/8/2022,142,"$4,359,000.00",2137,"$2,039.78",Yackeline Leiderman,NON-MLS MEMBER,15701 Collins Condo 15701 Collins Ave 1902 Sun...,,15701 Collins Condo 15701 Collins Ave 1902 Sun...,"(15701 Collins Ave #1902, North Miami Beach, F...","(25.9220146, -80.1217034, 0.0)",25.922015,-80.121703,0.0
7,The Oceansound\r251 Crandon Blvd 624\rKey Bisc...,11/29/2021,2/8/2022,17,"$2,700,000.00",3064,$881.20,Julie Snyder de Garcia,Lourdes Lorenzo-Luaces,The Oceansound 251 Crandon Blvd 624 Key Biscayne,The Oceansound,251 Crandon Blvd 624 Key Biscayne,"(251 Crandon Blvd APT 624, Key Biscayne, FL 33...","(25.6977929, -80.1579812, 0.0)",25.697793,-80.157981,0.0
8,Arbitare\r3165 Via Arbitare Way 4\rMiami,10/27/2021,2/8/2022,51,"$2,500,000.00",2872,$870.47,Javier Gonzalez PA,Riley Smith,Arbitare 3165 Via Arbitare Way 4 Miami,Arbitare,3165 Via Arbitare Way 4 Miami,"(3165 Via Abitare Way #4, Miami, FL 33133, USA...","(25.7250044, -80.2427871, 0.0)",25.725004,-80.242787,0.0
9,Country Lane Condo\r17000 NW 67 Ave 303\rHialeah,11/1/2021,2/9/2022,65,"$2,130,000.00",1038,"$2,052.02",Varda Peper,Sergio Mena,Country Lane Condo 17000 NW 67 Ave 303 Hialeah,Country Lane Condo,17000 NW 67 Ave 303 Hialeah,"(17000 NW 67th Ave APT 303, Hialeah, FL 33015,...","(25.929029, -80.31099309999999, 0.0)",25.929029,-80.310993,0.0
10,Portofino Tower Condo\r300 S Pointe Dr 1001\rM...,2/1/2021,2/8/2022,11,"$2,050,000.00",2340,$876.07,Desiree Prado,Christian Brandl,Portofino Tower Condo 300 S Pointe Dr 1001 Mia...,Portofino Tower Condo,300 S Pointe Dr 1001 Miami Beach,"(300 South Pointe Dr APT 1001, Miami Beach, FL...","(25.7677332, -80.1349941, 0.0)",25.767733,-80.134994,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
df.at[1,'building_name']=('Palazzo Del Mare')
df.at[1,'Agent']=('Alex Filkshteyn')
df.at[1,'Buyer Agent']=('Elena Bluntzer')

df.at[2,'Buyer Agent']=('Jonathan Mann')
df.at[2,'Agent']=('Elena Bluntzer & Jonathan Garcia')

In [11]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Buyer Agent',
       'geo_address', 'building_name', 'address_only', 'loc', 'point', 'lat',
       'lon', 'altitude'],
      dtype='object')

In [12]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('January 30th - February 5th')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [13]:
m.save('index.html')